In [1]:
import pandas as pd
import gmaps
import requests as rq
from config import g_key

In [2]:
cityDatadf = pd.read_csv("weather_data/cities.csv")

In [3]:
gmaps.configure(api_key=g_key)

In [9]:
locations = cityDatadf[["Lat", "Lng"]]
maxTemp = cityDatadf["Max Temp"]

#because gmaps does not plot negative numbers 
#(makes everything either the temp or 0 if too small)
temps = []
for temp in maxTemp:
    temps.append(max(temp, 0))
    
fig = gmaps.figure(center = (30.0, 31.0), zoom_level=1.5)
heatLayer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in maxTemp], dissipating=False, max_intensity=300, point_radius= 4)
fig.add_layer(heatLayer)

fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
locations = cityDatadf[["Lat", "Lng"]]
humidity = cityDatadf["Humidity"]
    
fig = gmaps.figure(center = (30.0, 31.0), zoom_level=1.5)
heatLayer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius= 4)
fig.add_layer(heatLayer)

fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
locations = cityDatadf[["Lat", "Lng"]]
clouds = cityDatadf["Cloudiness"]
    
fig = gmaps.figure(center = (30.0, 31.0), zoom_level=1.5)
heatLayer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius= 4)
fig.add_layer(heatLayer)

fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
locations = cityDatadf[["Lat", "Lng"]]
wind = cityDatadf["Wind Speed"]
    
fig = gmaps.figure(center = (30.0, 31.0), zoom_level=1.5)
heatLayer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius= 4)
fig.add_layer(heatLayer)

fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
minTemp = float(input("What is the minimum temp you would like for your trip? "))
maxTemp = float(input("What is the maximum temp you would like for your trip? "))

What is the minimum temp you would like for your trip? 75
What is the maximum temp you would like for your trip? 90


In [15]:
preferredCitieddf = cityDatadf.loc[(cityDatadf["Max Temp"] <= maxTemp) & (cityDatadf["Max Temp"] >= minTemp)]
preferredCitieddf.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
23,23,Dwarka,IN,2021-10-18 15:32:48,22.2394,68.9678,81.46,65,78,11.99
24,24,Tiarei,PF,2021-10-18 15:32:49,-17.5333,-149.3333,75.22,76,64,20.40
25,25,Vanimo,PG,2021-10-18 15:32:49,-2.6741,141.3028,80.08,87,99,6.71
27,27,Pangai,TO,2021-10-18 15:32:50,-19.8000,-174.3500,76.37,78,18,21.41
28,28,Oranjestad,AW,2021-10-18 15:31:39,12.5240,-70.0270,85.95,74,20,16.11


In [91]:
hoteldf = preferredCitieddf[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hoteldf["Hotel Name"] = ""
hoteldf.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
23,Dwarka,IN,81.46,22.2394,68.9678,
24,Tiarei,PF,75.22,-17.5333,-149.3333,
25,Vanimo,PG,80.08,-2.6741,141.3028,
27,Pangai,TO,76.37,-19.8000,-174.3500,
28,Oranjestad,AW,85.95,12.5240,-70.0270,


In [89]:
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

for index, row in hoteldf.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    baseurl = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = rq.get(baseurl, params=params).json()
    # Grab the first hotel from the results and store the name.
    testerdf = hotels["results"][0]["name"]
    testerdf
    try:
        hoteldf.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Did not work.")

IndexError: list index out of range

In [95]:
locations = hoteldf[["Lat","Lng"]]
maxTemp = hoteldf["Max Temp"]
fig = gmaps.figure(center = (30.0, 31.0), zoom_level=1.5)
heatLayer = gmaps.heatmap_layer(locations, weights=maxTemp, dissipating = False,
                               max_intensity = 300, point_radius=4)

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

hotelInfo = [info_box_template.format(**row) for index, row in hoteldf.iterrows()]
markerLayer = gmaps.marker_layer(locations, info_box_content=hotelInfo)

fig.add_layer(heatLayer)
fig.add_layer(markerLayer)
fig

Figure(layout=FigureLayout(height='420px'))